In [2]:
import pandas as pd
import numpy as np
import requests
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Input 
### PostgreSQL
Henter bilernes registreringsnummer fra flådestyrings tabel

In [7]:
user = ''
pw = ''
port = 5432
host = ''
db = ''

In [8]:
con = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, pw, host, port, db))

In [23]:
query = """
    SELECT bilreg 
    FROM lora_flaadestyring.bil_bilreg_euid 
    WHERE bilreg <> '*mangler*'
    """

In [24]:
bilreg = pd.read_sql_query(query,con=con)

In [30]:
plader = bilreg.bilreg.values

# Berig
### Motorregister API
Henter biloplysninger fra https://motorregisterapi.com/

In [36]:
auth = ''
url = 'https://motorregisterapi.com/api/vehicles/get/'

In [37]:
data = []
for plade in plader:
    url = 'https://motorregisterapi.com/api/vehicles/get/{}'.format(plade)
    r = requests.get(url, auth=(auth, auth))
    d = r.json()
    data.append(d)

In [63]:
df = pd.DataFrame(data)

Fjerner rækker hvor der ikke blev fundet oplysninger på nummerpladen 

In [65]:
df.dropna(axis=0, subset=['LicensePlateNumber'], inplace=True)

In [66]:
df.set_index('LicensePlateNumber', inplace=True)

In [67]:
df

,LeasingValidFrom,LeasingValidTo,LicensePlateExpirationDate,LicensePlateValidFrom,LicensePlateValidTo,Message,VehicleEnvironmentCO2Release,VehicleEnvironmentEmissionCO,VehicleEnvironmentEmissionHCPlusNOX,VehicleEnvironmentEmissionNOX,...,VehicleMotorMarking,VehicleMotorMileage,VehicleMotorMileageDocumentation,VehicleMotorPlugInHybrid,VehicleMotorRunningSoundLevel,VehiclePostalCode,VehicleRegistrationDate,VehicleRegistrationStatus,VehicleUsageName,VehicleUsageNumber
LicensePlateNumber,,,,,,,,,,,,,,,,,,,,,
ZM37183,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,0,0,0,0,...,None,91,False,False,0,None,2008-08-13T22:00:00,Registreret,Privat personkørsel,1
BT60163,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2017-11-16T06:54:08,Registreret,Privat personkørsel,1
BS15899,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2018-01-15T09:40:13,Registreret,Privat personkørsel,1
BL55280,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2018-01-15T09:38:09,Registreret,Privat personkørsel,1
BR93131,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2017-11-10T08:30:43,Registreret,Privat personkørsel,1
BV14903,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2017-12-11T06:50:26,Registreret,Privat personkørsel,1
BS15898,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2018-01-15T09:41:24,Registreret,Privat personkørsel,1
BV14900,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2017-12-11T06:27:43,Registreret,Privat personkørsel,1
BV14901,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,NaN,96,482.7,0,17.8,...,*CHY??????*,None,False,False,69,2750,2017-12-11T06:34:07,Registreret,Privat personkørsel,1


# Output
Gemmer i PostgreSQL database

In [68]:
table = 'biler'
schema = 'lora_flaadestyring'

In [69]:
df.to_sql(table, con, schema=schema, if_exists='replace')

# Gammelt

### T Hansen API

In [54]:
token = '18506180474'
plader = 'ZM37183 BT60163 BS15899 BL55280 BR93131 BV14903 BS15898 BV14900 BV14901 BZ55107 BZ55115 BZ55106'.split(' ')

In [55]:
data = []

In [56]:
for plade in plader:
    url = 'https://dmr.thg.dk/importoer/api/Koeretoejekst/GetTekniskData?token={}&plade={}'.format(token, plade)
    r = requests.get(url)
    d = r.json()
    data.append(d)

In [57]:
df = pd.DataFrame(data)

In [58]:
df.set_index('RegNr', inplace=True)